In [62]:
load('aca2.mat')
X = transpose(X);
s = transpose(s);

In [63]:
size(X)


ans =

        2204          42



In [64]:
size(s)


ans =

        2204           1



In [65]:
a = [1 2 3; 4 5 6];
a_row_norm = sqrt(sum(a.^2,2));
a_normalized = bsxfun(@rdivide, a, a_row_norm); 
sqrt(sum(a_normalized.^2,2))



ans =

    1.0000
    1.0000



In [66]:
X_truncated = X(1:2:end, :);
s_truncated = s(1:2:end);


In [67]:
X_col_norm = sqrt(sum(X_truncated.^2,1)); 
X_truncated = bsxfun(@rdivide,X_truncated, X_col_norm); 
X_col_norm = reshape(X_col_norm,[],1);
X_truncated(isnan(X_truncated)) = 0;


In [ ]:
gamma =1 ;
k = 2;
min_error =  1e9;
min_gamma = 1;
min_k = 2;

errors_laplacian_unnormalized = zeros(100, 49);
errors_laplacian_normalized = zeros(100, 49);
errors_laplacian_rw = zeros(100, 49);

errors_minimum = zeros(100, 49);
errors_minimum_index = zeros(100, 49);

for gamma=1:100    
    for k=2:50
       [idx1, idx2, idx3] = SpectralClustering(gamma, k, X_truncated);
       missclassification_error = Misclassification([idx1 idx2 idx3], s_truncated);
       [min_value, min_index] = min(missclassification_error);
       if min_value < min_error
           min_error = min_value;
           min_gamma = gamma;
           min_k = k;
        end
       errors_laplacian_unnormalized(gamma, k-1) =  missclassification_error(1);
       errors_laplacian_normalized(gamma, k-1) =  missclassification_error(2);
       errors_laplacian_rw(gamma, k-1) =  missclassification_error(3);
       
       errors_minimum(gamma, k-1) = min_value;
       errors_minimum_index(gamma, k-1) = min_index;
    end     
end
    

In [80]:
x = containers.Map();
x('test') = [1 2 3]


x = 

  Map with properties:

        Count: 1
      KeyType: char
    ValueType: any



In [83]:
x('test')


ans =

     1     2     3



In [76]:
min_error


min_error =

    0.2250



In [77]:
min_k


min_k =

     9



In [78]:
min_gamma


min_gamma =

    16



In [75]:
missclassification_error(1)


ans =

    0.8448



In [43]:

K = zeros(size(X_truncated, 1), size(X_truncated, 1));
for i=1:size(X_truncated,1)
    for j=1:size(X_truncated,1)
        K(i,j) = exp(-gamma* (norm( X_truncated(i, :) - X_truncated(j, :), 2).^2) );
    end
end     

In [45]:
K_maxk = maxk(K, k);
W = K;
for i=1:size(X_truncated,1)
    for j=1:size(X_truncated,1)
        if K(i,j) < min(K_maxk(:, j))
            W(i,j) = 0;
        end
    end
end
W = (W+transpose(W))/2;
D = diag(W*ones(size(W,1),1));
I = eye(size(W,1));
laplacian_unnormalized = D-W;
laplacian_normalized = I-W;
laplacian_rw = I-inv(D)*W;




In [51]:
[V1, D1] = eigs(laplacian_unnormalized, k);
[V2, D2] = eigs(laplacian_normalized, k);
[V3, D3] = eigs(laplacian_rw, k);


In [52]:
V1_row_norm = sqrt(sum(V1.^2,2));
Y1 = bsxfun(@rdivide, V1, V1_row_norm);

V2_row_norm = sqrt(sum(V2.^2,2));
Y2 = bsxfun(@rdivide, V2, V2_row_norm);

V3_row_norm = sqrt(sum(V3.^2,2));
Y3 = bsxfun(@rdivide, V3, V3_row_norm);


In [53]:
idx1 = kmeans(Y1, k);
idx2 = kmeans(Y2, k);
idx3 = kmeans(Y3, k);

In [56]:
size(idx1)


ans =

        1102           1



In [57]:
Misclassification([idx1 idx2 idx3], s_truncated)


ans =

    0.7740
    0.7541
    0.7713

